# Chargement des données et séparation des features et target

Avant tout préprocessing, on établit la séparation des données de train et de test afin de limiter la possibilité de data leak entre les jeux de données de train et de test. Nous avons choisi le ratio standard suivant : 80% de train, 20% de test.

On sait que la classe 2583 (accessoires de piscines) est sur représentée dans le jeu de données. Nous avons décidé de :
- faire de l'undersampling aléatoire pour diminuer sa présence à 3% au lieu de 12%
- faire de l'oversampling aléatoire à 3% pour les classes minoritaires

Cela permet d'équilibrer les classes présentes et améliorer les performances de l'apprentissage.

#### Proportion de chaque classe dans les jeux de données

| Classe | Train avant resampling | Train après resampling | Test      |
|--------|------------------------|------------------------|-----------|
| 2583   | 12.022316              | 3.703704               | 12.023081 |
| 1560   | 5.973621               | 3.703704               | 5.976213  |
| 1300   | 5.941235               | 3.703704               | 5.940886  |
| 2060   | 5.879409               | 3.703704               | 5.882007  |
| 2522   | 5.874993               | 3.703704               | 5.876119  |
| 1280   | 5.735147               | 3.703704               | 5.734809  |
| 2403   | 5.621798               | 3.703704               | 5.622939  |
| 2280   | 5.605606               | 3.703704               | 5.605276  |
| 1920   | 5.066832               | 3.703704               | 5.069477  |
| 1160   | 4.654655               | 3.703704               | 4.657325  |
| 1320   | 3.817052               | 3.703704               | 3.815356  |
| 10     | 3.669846               | 3.703704               | 3.668158  |
| 2705   | 3.251781               | 3.703704               | 3.250118  |
| 1140   | 3.145793               | 3.703704               | 3.144136  |
| 2582   | 3.048637               | 3.703704               | 3.049929  |
| 40     | 2.952953               | 3.703704               | 2.955723  |
| 2585   | 2.939704               | 3.703704               | 2.938059  |
| 1302   | 2.933816               | 3.703704               | 2.932171  |
| 1281   | 2.437732               | 3.703704               | 2.437588  |
| 50     | 1.979921               | 3.703704               | 1.978333  |
| 2462   | 1.673733               | 3.703704               | 1.672162  |
| 2905   | 1.027498               | 3.703704               | 1.024494  |
| 60     | 0.980392               | 3.703704               | 0.977390  |
| 2220   | 0.970088               | 3.703704               | 0.971503  |
| 1301   | 0.950951               | 3.703704               | 0.947951  |
| 1940   | 0.945063               | 3.703704               | 0.947951  |
| 1180   | 0.899429               | 3.703704               | 0.900848  |

# Preprocessing des données

## Preprocessing des données textuelles

### Création de la variable `has_description` basée sur la présence de la variable `description`

Nous avons vu lors de l'exploration que la répartition de la présence de description n'est pas homogène parmi les catégories de produits. On peut en déduire que l'absence ou présence de description est une information à part entière qu'il peut être intéressant d'exploiter dans nos modèles.

Nous décidons de créer la variable `has_description` qui vaut :
- 1 si la description est présente
- 0 sinon

### Fusion des variables `designation` et `description` dans `full_description`

La `description` et la `designation` sont toutes les deux des valeurs textuelles libres. Bien que `designation` soit plus courte, elles apportent le même type d'information sur le produit c'est-à-dire sa description et son appellation commune. `description` pouvant être nulle, cela peut être gênant pour son intégration dans les modèles. 

Afin de faciliter le traitement de ces deux valeurs textuelles, nous décidons de les fusionner dans une variable nommée `full_description`

### Nettoyage, tokenisation et stemmatisation de la variable `full_description`

Nous souhaitons faire les étapes de préprocessing suivantes :

- Suppression des stop words (anglais et français)
- Suppression des ponctuations
- Suppression des balises HTML
- Stemmatisation
- Application d'un TF IDF

Après quelques tentatives de création d'un process, nous décidons d'abandonner la stemmatisation. On note avec l'extrait ci-dessous que la stemmatisation est trop violente sur le contenu de la variable d'une part et qu'il n'est pas adapté sur un contenu multilingue tel que nous avons ici (français, anglais mais aussi parfois allemand). Afin de limiter la perte en information, nous décidons de ne pas appliquer de lemmatisation ni de stemmatisation.

| preprocessed_full_description |                                  full_description |                                                   |
|------------------------------:|--------------------------------------------------:|---------------------------------------------------|
|                             0 |                              ship focus record 31 |                          Ships In Focus Record 31 |
|                             1 | convergent divergent alignment standardsassess... | Convergence Or Divergence: Alignment Of Standa... |
|                             2 | lecon scienc cour superieur premier anne certi... | Lecons De Sciences - Cours Superieur - Premier... |
|                             3 |                             gener women historian |                   Generations Of Women Historians |
|                             4 | magnet encycloped spiritual trait spécial fait... | Magnétisme: Encyclopédie Magnétique Spirituali... |
|                             5 | inglorious art peac exhibit canadian society n... | The Inglorious Arts Of Peace: Exhibitions In C... |
|                             6 |              sylvi souvient-il vi giacomo leopard | Sylvia Te Souvient-Il ? - Vie De Giacomo Leopardi |
|                             7 |                             trachom malad pauvret |          Le Trachome - Une Maladie De La Pauvreté |
|                             8 |                   business associ 2006 statut rul |    Business Associations: 2006 Statutes And Rules |
|                             9 |                        luun tom 8 l'attrapeur rêv |               Luuna Tome 8 - L'attrapeur De Rêves |

Il aurait aussi été possible de traduire les textes pour uniformiser la langue employée mais cela représente :
- un traitement coûteux
- un risque de perte d'information si la traduction n'est pas de bonne qualité

Comme le jeu de données est conséquent, on peut s'attendre à ce que les algorithme d'analyse textuel arrivent à extraire l'information utile malgré la présence de multiple langues.

On décide donc de limiter le prétraitement textuel aux éléments suivants :
- nettoyage des stopwords anglais et français
- nettoyage de la ponctuation
- tokenisation des chaînes de caractères

### Application de TF-IDF à la variable `full_description`

En l'état, les chaînes de caractères dans full_description sont difficilement exploitables par des algorithmes de machine learning.

Il est nécessaire de les transformer en valeurs numériques interprétables. Nous avons décidé d'appliquer le TF-IDF pour une extraction de donnée rapide et efficace suite à notre analyse en wordcloud qui fait apparaître la répétition de mots clés pour chaque catégorie de produits.

### Sauvegarde des données textuelles suite à leur préprocessing

Afin de faciliter le travail sur plusieurs modèles et gagner du temps, on décide de sauvegarder les données prétraitées.

## Preprocessing des données graphiques